### Merging NED13 tiles and generating hillshaded color-relief terrain maps

In [ ]:
import os
import sys
import json
import glob
import datetime
import rasterio
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

In [ ]:
# load the raw NED13 tile directories
ned13_dirs = [p for p in glob.glob('/media/keith/USGS_Backup/USGS/NED13/*') if os.path.isdir(p)]
ned13_dirs.sort()

In [ ]:
sys.path.append('../../../_projects-db/sierra-map/src/')
import gdal_wrappers as gw

In [ ]:
# downsample and reproject tiles as tifs
# (useful for quickly testing various ROIs)
for path in ned13_dirs:
    d = managers.datasets.new_dataset('ned13', path)
    print(d.name)
    
    output_filename = '../test/datasets/dem/NED13-tiles/%s_100m.tif' % d.name
    if os.path.isfile(output_filename):
        continue
    
    gw.warp(d.bandpath(), output_filename, r='bilinear', tr=(100, 100), t_srs='EPSG:3857')

In [ ]:
# all of the downsampled tiles
dataset_paths = glob.glob('../test/datasets/dem/NED13-tiles/*.tif')

In [ ]:
# manually-defined ROIs
bounds = {
    'guerneville': [-123.8, 38.4, -122.8, 39.],
    'south-bay': [-122.55, 36.93, -121.75, 37.65],
    'berkeley': [-122.38, 37.75, -122.06, 38.0],
    'sf': [-122.54, 37.64, -122.34, 37.81],
}

In [ ]:
# original sierra map cmap
cmap_orig = [
    {'elevation': 0.0, 'color': (25, 125, 225)},
    {'elevation': 3.28, 'color': (110, 140, 100)},
    {'elevation': 328.0, 'color': (115, 150, 105)},
    {'elevation': 1312.0, 'color': (140, 155, 115)},
    {'elevation': 1640.0, 'color': (190, 204, 145)},
    {'elevation': 3280.0, 'color': (250, 250, 185)},
    {'elevation': 4920.0, 'color': (250, 205, 160)},
    {'elevation': 6560.0, 'color': (230, 180, 155)},
    {'elevation': 8200.0, 'color': (240, 200, 190)},
    {'elevation': 9840.0, 'color': (245, 220, 210)},
    {'elevation': 11480.0, 'color': (253, 235, 230)},
    {'elevation': 13120.0, 'color': (255, 255, 255)}]

In [ ]:
# cmap adapted from the original sierra map (elevations in feet)
norcal_cmap = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 100, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 400, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 500, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 1000, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 1500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 2000, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 2500, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 3000, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 3500, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 3600, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
# same as norcal_cmap but with less abrupt green-yellow transition at low elevations
norcal_cmap_2 = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 300, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 600, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 900, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 1200, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 1500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 1800, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 2100, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 2400, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 2700, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 3000, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
cmap_for_sf = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 100, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 200, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 300, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 400, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 600, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 700, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 800, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 900, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 1000, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
sns.palplot([np.array(row['color'])/255. for row in norcal_cmap])

In [ ]:
def workflow(project_root, colormap, bounds=None, res=None, reset=False):
    '''
    Workflow for cropping/merging and processing NED13 ROIs
    '''

    # transform bounds from lat-lon to the NED13 CRS
    if bounds:
        d = managers.datasets.new_dataset('ned13', ned13_dirs[0], exists=True)
        bounds = managers.utils.transform(bounds, d.bandpath())

    proj = managers.DEMProject(
        project_root=project_root, 
        dataset_paths=ned13_dirs,
        bounds=bounds,
        res=res,
        reset=reset)
    
    # note that using get_operation('last') here assumes reset=True
    proj.warp(proj.get_operation('last'), crs='EPSG:3857', res=None)
    proj.hill_shade(proj.get_operation('last', 'warp'))
    proj.color_relief(proj.get_operation('last', 'warp'), colormap=colormap)
    proj.multiply([proj.get_operation('last', 'hill_shade'), proj.get_operation('last', 'color_relief')])
    proj.save_props()

In [ ]:
# merge and crop all raw NED13 tiles
project_root='/home/keith/raster-projects/ned13-all/'
workflow(project_root, bounds=None, res=5e-3, reset=True, colormap=norcal_cmap_2)

In [ ]:
# merge and crop for SF
project_root = '/home/keith/raster-projects/ned13-sf/'
workflow(project_root, cmap_for_sf, bounds=bounds['sf'], res=None, reset=True)

### Debugging

In [ ]:
proj = managers.DEMProject(project_root='/home/keith/raster-projects/ned13-downsampled-norcal/')
with rasterio.open(proj.get_operation('last', 'hillshade').destination.path) as src:
    im_ds = src.read()

In [ ]:
proj = managers.DEMProject(project_root='/home/keith/raster-projects/ned13-berkeley/', reset=False)

In [ ]:
proj.texture_shade(proj.get_operation('last', 'warp'))

In [ ]:
proj.multiply([proj.get_operation('first', 'multiply'), proj.get_operation('last', 'texture_shade')], weight=.5)

In [ ]:
proj._serialize()